Obtain vibrational energy levels and rotational temperatures for some small molecules from QM calculation (e.g. using Gaussian); calculate their Cv at various temperatures and compare with tabulated values.

Only the calculation for the first molecule (in this case CO2), will be described in detail to illustrate the workflow.

Gaussian job setup for a frequency calculation:

```
%chk=l04_qmcalc.chk
#p opt freq ccsd/cc-pvtz

opt + freq on co2

0 1
C  0.0  0.0  0.0
O -1.0  0.0  0.0
O  1.0  0.0  0.0
```

Frequency information retrieved from the log file of the finished calculation:
```
 Harmonic frequencies (cm**-1), IR intensities (KM/Mole), Raman scattering
 activities (A**4/AMU), depolarization ratios for plane and unpolarized
 incident light, reduced masses (AMU), force constants (mDyne/A),
 and normal coordinates:
                      1                      2                      3
                     PIU                    PIU                    SGG
 Frequencies --    684.5137               684.5137              1389.7091
 Red. masses --     12.8774                12.8774                15.9949
 Frc consts  --      3.5550                 3.5550                18.2004
 IR Inten    --     34.3245                34.3245                 0.0000
  Atom  AN      X      Y      Z        X      Y      Z        X      Y      Z
     1   6     0.00   0.88   0.00     0.88  -0.00   0.00     0.00  -0.00  -0.00
     2   8    -0.00  -0.33   0.00    -0.33   0.00  -0.00    -0.00   0.00   0.71
     3   8    -0.00  -0.33  -0.00    -0.33   0.00  -0.00     0.00   0.00  -0.71
                      4
                     SGU
 Frequencies --   2434.6980
 Red. masses --     12.8774
 Frc consts  --     44.9746
 IR Inten    --    709.2532
  Atom  AN      X      Y      Z
     1   6    -0.00  -0.00   0.88
     2   8     0.00   0.00  -0.33
     3   8     0.00   0.00  -0.33
```

From the above we can see that CO2 has 4 vibrational modes, two of which are degenerate bending modes ($684 \; cm^{-1}$), one is a symmetric strech ($1389 \; cm^{-1}$) and the other is an antisymmetric strech ($2434 \; cm^{-1}$)

Thermochemical information such as rotational temperatures can be retrieved from the Thermochemistry section of the Gaussian log:
```
 -------------------
 - Thermochemistry -
 -------------------
 Temperature   298.150 Kelvin.  Pressure   1.00000 Atm.
 Atom     1 has atomic number  6 and mass  12.00000
 Atom     2 has atomic number  8 and mass  15.99491
 Atom     3 has atomic number  8 and mass  15.99491
 Molecular mass:    43.98983 amu.
 Principal axes and moments of inertia in atomic units:
                           1         2         3
     Eigenvalues --     0.00000 153.49903 153.49903
           X           -0.00000   1.00000   0.00000
           Y           -0.00000   0.00000   1.00000
           Z            1.00000   0.00000   0.00000
 This molecule is a prolate symmetric top.
 Rotational symmetry number  2.
 Rotational temperature (Kelvin)      0.56426
 Rotational constant (GHZ):          11.757346
 Zero-point vibrational energy      31063.6 (Joules/Mol)
                                    7.42439 (Kcal/Mol)
 Vibrational temperatures:    984.86   984.86  1999.48  3502.99
          (Kelvin)
 
 Zero-point correction=                           0.011832 (Hartree/Particle)
 Thermal correction to Energy=                    0.014438
 Thermal correction to Enthalpy=                  0.015382
 Thermal correction to Gibbs Free Energy=        -0.008854
 Sum of electronic and zero-point Energies=           -188.286840
 Sum of electronic and thermal Energies=              -188.284234
 Sum of electronic and thermal Enthalpies=            -188.283290
 Sum of electronic and thermal Free Energies=         -188.307525
 
                     E (Thermal)             CV                S
                      KCal/Mol        Cal/Mol-Kelvin    Cal/Mol-Kelvin
 Total                    9.060              6.798             51.008
 Electronic               0.000              0.000              0.000
 Translational            0.889              2.981             37.270
 Rotational               0.592              1.987             13.069
 Vibrational              7.579              1.830              0.669
                       Q            Log10(Q)             Ln(Q)
 Total Bot       0.118130D+05          4.072360          9.376956
 Total V=0       0.326945D+10          9.514474         21.907887
 Vib (Bot)       0.389902D-05         -5.409045        -12.454786
 Vib (V=0)       0.107912D+01          0.033069          0.076145
 Electronic      0.100000D+01          0.000000          0.000000
 Translational   0.114679D+08          7.059482         16.255059
 Rotational      0.264194D+03          2.421923          5.576683
```

Where rotational temperature is 0.56 Kelvin

Heat capacity at constant volume is the derivative of internal energy with respect to temperature:

$C_V = \frac{\partial{U}}{\partial{T}}$

Since we are only interested in the vibrational contribution to heat capacity:

$C_{V,vib} = \frac{\partial{E_{vib}}}{\partial{T}}$

Assuming a quantum harmonic oscillator, energy of a vibrational mode is:

$E_n = h \nu (n + \frac{1}{2})$

$C_V = \frac{\partial{U}}{\partial{T}}$

$C_{V,vib} = \frac{\partial{E_{vib}}}{\partial{T}}$

$E_n = h \nu (n + \frac{1}{2})$

$Q=\sum_i e^{-\beta E_n}$

$\beta = \frac{1}{kT}$

$C_V = R \sum_K e^{\Theta_{v,K}/T} \Big( \frac{\Theta_{v,K}/T}{e^{-\Theta_{v,K}/T} - 1} \Big)$

In [3]:
import numpy as np
from scipy import constants

def calc_theta(frequencies):
    '''Calculation of characteristic vibrational temperatures of oscillator modes'''
    theta_all = []
    for frequency in frequencies:
        theta_k = (constants.Planck * frequency ) / constants.Boltzmann
        theta_all.append(theta_k)
    return theta_all

def vibrational_heat_capacity(theta, temp):
    '''Calculation of isochoric vibrational heat capacity'''
    intermediate = 0
    vib_cv_modes = []
    #sum
    for k in range(len(theta)):
        intermediate += np.exp(theta[k] / temp) * ( (theta[k] / temp) / (np.exp(theta[k] / temp) - 1) )
        vib_cv_of_mode = constants.gas_constant * np.exp(theta[k] / temp) * ( (theta[k] / temp) / (np.exp(theta[k] / temp) - 1) )
        vib_cv_modes.append(vib_cv_of_mode)

    total_vib_cv = constants.gas_constant * intermediate
    return vib_cv_modes, total_vib_cv

frequencies = [684, 1389, 2343]
temp = 298.15

theta = calc_theta(frequencies)
vib_cv_of_modes, total_vib_contribution = vibrational_heat_capacity(theta, temp)

print(frequencies)
print(theta)
print(vib_cv_of_modes)
print(total_vib_contribution)

[684, 1389, 2343]
[3.282682262182495e-08, 6.66614862890568e-08, 1.1244626520897055e-07]
[8.314462395496752, 8.314458992146468, 8.314464501764414]
24.943385889407633
